# Stackelberg Economy

Imports and set magics:

In [345]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact



# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Once again I have been unable to get the py file to work. I dont know why this is. Thus, all functions are included in the notebook. I apologize for this

# Model description

I will be looking at a Stackelberg Economy where two firms compete by producing identical goods. To begin, Firm 1 chooses an amount q1. After observing q1, Firm 2 chooses an amount q2. The demand curve is linear and takes the form:

$ p(q) = a - q $ 

where $ q = q1 + q2 $. Furthermore, $ p(q) = 0 $ for $ q > a $

Each firm faces a marginal cost of $ c_i $

Thus, each firm aims to maximize profit given by

$ p(q) * q_i - c_i * q_i $


## Analytical solution

In [346]:
p = sm.symbols('p')
q = sm.symbols('q')
q1 = sm.symbols('q_1')
q2 = sm.symbols('q_2')
c1 = sm.symbols('c_1')
c2 = sm.symbols('c_2')
a = sm.symbols('a')
s = sm.symbols('s')

In [347]:
# Defining objective functions
objective1 = p * q1 - c1 * q1
objective2 = p * q2 - c2 * q2

# Defining price function
price_func = sm.Eq(p, a - q)

# Defining total quantity function
total_q = sm.Eq(q, q1 + q2)

In [348]:
# Substituting total quantity into the price function
price_func1 = price_func.subs(q, total_q.rhs)

# Subsituting the price function into firm 2's objective function
objective2_sub = objective2.subs(p, price_func1.rhs)

# Differentiating firm 2's objective function wrt q2
foc2 = sm.diff(objective2_sub, q2)

# Setting the FOC equal to 0 and solving for q2
sol_q2 = sm.solve(sm.Eq(foc2, 0), q2)
sol_q2[0]



a/2 - c_2/2 - q_1/2

In [349]:
# Substituting the price function into firm 1's objective function
objective1_sub = objective1.subs(p, price_func1.rhs)

# Substituting firm 2's best response into firm 1's objective function
objective1_sub2 = objective1_sub.subs(q2, sol_q2[0])

# Differentiating firm 1's objective function wrt q1
foc1 = sm.diff(objective1_sub2, q1)

# Setting the FOC equal to 0 and solving for q1
sol_q1 = sm.solve(sm.Eq(foc1, 0), q1)
sol_q1[0]


a/2 - c_1 + c_2/2

In [350]:
# Substituting q1 back into q2 to find firm 2's quantity independet of q1
sol_q2_final = sol_q2[0].subs(q1, sol_q1[0])
sol_q2_final


a/4 + c_1/2 - 3*c_2/4

In [351]:
# Subsituting q1 and q2 into the price function the get the market price
price_func2 = price_func1.subs(q1, sol_q1[0])
price_func3 = price_func2.subs(q2, sol_q2_final)
price_func3



Eq(p, a/4 + c_1/2 + c_2/4)

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

In [352]:
objective2_sub

-c_2*q_2 + q_2*(a - q_1 - q_2)

**Model-Solving Algorithm**

In [353]:
#Define model-solving function
def modelsolving(a, c1, c2):
    # Define objective function
    def objective_function1(q1, a, c1, c2):
        return -(-c1*q1 + (q1/2) * (a + c2 - q1))

    # Define an initial guess for q1
    initial_guess_q1 = 2.0
    bounds = [(0, None)]

    # Call the minimize function
    result_q1 = optimize.minimize(objective_function1, initial_guess_q1,
                            args=(a, c1, c2),
                            method='SLSQP',
                            bounds=bounds,
                            options={'disp': False})

    # Print the result
    #print(result_q1.x[0])
    q1, pi1 = result_q1.x[0], -result_q1.fun

    def objective_function2(q2, a, c1, c2, q1):
        return -(-c2*q2 + q2*(a - q1 - q2))

    result_q2 = optimize.minimize(objective_function2, initial_guess_q1,
                            args=(a, c1, c2, q1),
                            method='SLSQP',
                            bounds=bounds,
                            options={'disp': False})

    print(result_q2)
    q2, pi2 = result_q2.x[0], -result_q2.fun

    return q1, q2, pi1, pi2



In [354]:
# Setting Parameters
a = 14
c1 = 5
c2 = 4

# Solving Model
q1, q2, pi1, pi2 = modelsolving(a, c1, c2)
price = a - q1 - q2

print(f"Firm 1 produces {q1} units and has a profit of {pi1}")
print(f"Firm 2 produces {q2} units and has a profit of {pi2}")
print(f"The price of one unit is {price}")

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -9.0
       x: [ 3.000e+00]
     nit: 2
     jac: [ 0.000e+00]
    nfev: 5
    njev: 2
Firm 1 produces 4.0 units and has a profit of 8.0
Firm 2 produces 3.0 units and has a profit of 9.0
The price of one unit is 7.0


# Further analysis

First, I will illustrate the model using bar graphs for quantity and profit

In [418]:
def plot_quantity_bar_chart(initial_a, initial_c1, initial_c2):
    # Define initial values for a, c1, and c2
    a = initial_a
    c1 = initial_c1
    c2 = initial_c2
    
    q1, q2, pi1, pi2 = modelsolving(a, c1, c2)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the bar chart for quantities produced by Firm 1 and Firm 2
    bars1 = ax1.bar(['Firm 1', 'Firm 2'], [q1, q2], color=['blue', 'green'])
    ax1.set_ylabel('Quantity Produced')
    ax1.set_title('Quantities Produced by Firm 1 and Firm 2')

    # Add text labels on top of each bar in the quantity bar chart
    for bar in bars1:
        height = bar.get_height()
        ax1.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3),  # 3 points vertical offset
                     textcoords="offset points",
                     ha='center', va='bottom')
        
    # Plot the bar chart for profits of Firm 1 and Firm 2
    bars2 = ax2.bar(['Firm 1', 'Firm 2'], [pi1, pi2], color=['blue', 'green'])
    ax2.set_ylabel('Profit')
    ax2.set_title('Profits of Firm 1 and Firm 2')

    for bar in bars2:
        height = bar.get_height()
        ax2.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3),  # 3 points vertical offset
                     textcoords="offset points",
                     ha='center', va='bottom')
        
    
    
    plt.show()
    
    price = a - q1 - q2
    print(f"Unit Price: {price}")


# Create interactive sliders for changing initial values of a, c1, and c2
interact(plot_quantity_bar_chart, initial_a=widgets.FloatSlider(value=14, min=0, max=20, step=1), 
         initial_c1=widgets.FloatSlider(value=5, min=0, max=10, step=1),
         initial_c2=widgets.FloatSlider(value=4, min=0, max=10, step=1))

interactive(children=(FloatSlider(value=14.0, description='initial_a', max=20.0, step=1.0), FloatSlider(value=…

<function __main__.plot_quantity_bar_chart(initial_a, initial_c1, initial_c2)>

# Model Extension

As an extension, I will look at how this market is changed if Firm 1 already has a fixed stock of good which it needs to sell.

**Analytical Solution**

In [356]:
p = sm.symbols('p')
q = sm.symbols('q')
q1 = sm.symbols('q_1')
q2 = sm.symbols('q_2')
c1 = sm.symbols('c_1')
c2 = sm.symbols('c_2')
a = sm.symbols('a')
s = sm.symbols('s')

In [357]:
# Defining objective functions
objective1 = p * (s + q1) - c1 * q1
objective2 = p * q2 - c2 * q2

# Defining price function
price_func = sm.Eq(p, a - q)

# Defining total quantity function
total_q = sm.Eq(q, s + q1 + q2)

# Substituting total quantity into the price function
price_func1 = price_func.subs(q, total_q.rhs)

In [358]:
# Subsituting the price function into firm 2's objective function
objective2_sub = objective2.subs(p, price_func1.rhs)
objective2_sub

-c_2*q_2 + q_2*(a - q_1 - q_2 - s)

In [359]:
# Subsituting the price function into firm 2's objective function
objective2_sub = objective2.subs(p, price_func1.rhs)

# Differentiating firm 2's objective function wrt q2
foc2 = sm.diff(objective2_sub, q2)

# Setting the FOC equal to 0 and solving for q2
sol_q2 = sm.solve(sm.Eq(foc2, 0), q2)
sol_q2[0]

a/2 - c_2/2 - q_1/2 - s/2

In [360]:
# Substituting the price function into firm 1's objective function
objective1_sub = objective1.subs(p, price_func1.rhs)

# Substituting firm 2's best response into firm 1's objective function
objective1_sub2 = objective1_sub.subs(q2, sol_q2[0])

objective1_sub

-c_1*q_1 + (q_1 + s)*(a - q_1 - q_2 - s)

In [361]:

# Differentiating firm 1's objective function wrt q1
foc1 = sm.diff(objective1_sub2, q1)

# Setting the FOC equal to 0 and solving for q1
sol_q1 = sm.solve(sm.Eq(foc1, 0), q1)
sol_q1[0]

a/2 - c_1 + c_2/2 - s

In [362]:
# Substituting q1 back into q2 to find firm 2's quantity independent of q1
sol_q2_final = sol_q2[0].subs(q1, sol_q1[0])
sol_q2_final



a/4 + c_1/2 - 3*c_2/4

In [363]:

# Subsituting q1 and q2 into the price function the get the market price
price_func2 = price_func1.subs(q1, sol_q1[0])
price_func3 = price_func2.subs(q2, sol_q2_final)
price_func3

Eq(p, a/4 + c_1/2 + c_2/4)

**Numerical Solution**

In [383]:
#Define model-solving function
def modelsolving2(a, c1, c2, s):
    # Define objective function
    def objective_function1(q1, a, c1, c2, s):
        return -(-c1*q1 + ((s + q1)/2) * (a + c2 - q1 - s))

    # Define an initial guess for q1
    initial_guess_q1 = 2.0
    bounds = [(0, None)]

    # Call the minimize function
    result_q1 = optimize.minimize(objective_function1, initial_guess_q1,
                            args=(a, c1, c2, s),
                            method='SLSQP',
                            bounds=bounds,
                            options={'disp': False})

    # Print the result
    #print(result_q1.x[0])
    q1 = result_q1.x[0]

    def objective_function2(q2, a, c1, c2, q1):
        return -(-c2*q2 + q2*(a - q1 - q2))

    result_q2 = optimize.minimize(objective_function2, initial_guess_q1,
                            args=(a, c1, c2, s+q1),
                            method='SLSQP',
                            bounds=bounds,
                            options={'disp': False})

    print(result_q2)

    q2 = result_q2.x[0]
    price = max(0, a - s - q1 - q2)
    
    pi1, pi2 = (s + q1) * price - c1 * q1, q2 * price - q2 * c2

    return q1, q2, pi1, pi2, price


In [409]:
# Setting Parameters
a = 14
c1 = 5
c2 = 4
s = 4 

# Solving Model
q1, q2, pi1, pi2, price = modelsolving2(a, c1, c2, s)

print(f"Firm 1 produces {q1} units and has a profit of {pi1}")
print(f"Firm 2 produces {q2} units and has a profit of {pi2}")
print(f"The price of one unit is {price}")

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -9.0
       x: [ 3.000e+00]
     nit: 2
     jac: [ 0.000e+00]
    nfev: 5
    njev: 2
Firm 1 produces 0.0 units and has a profit of 28.0
Firm 2 produces 3.0 units and has a profit of 9.0
The price of one unit is 7.0


**Graphical Illustration**

In [417]:
# Define the function to plot the bar charts
def plot_quantity_bar_chart2(initial_a, initial_c1, initial_c2, initial_s):
    # Define initial values for a, c1, c2, and s
    a = initial_a
    c1 = initial_c1
    c2 = initial_c2
    s = initial_s
    
    q1, q2, pi1, pi2, price = modelsolving2(a, c1, c2, s)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the bar chart for quantities produced by Firm 1 and Firm 2
    bars1 = ax1.bar(['Firm 1', 'Firm 2'],  [q1, q2], color=['blue', 'green'])
    ax1.set_ylabel('Quantity Produced')
    ax1.set_title('Quantities Produced by Firm 1 and Firm 2')

    # Set y-axis lower limit to one higher than the maximum of q1 and q2
    ax1.set_ylim([0, max(s+q1, q2) + 1])


    # Add text labels on top of each bar in the quantity bar chart
    for bar in bars1:
        height = bar.get_height()
        ax1.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3),  # 3 points vertical offset
                     textcoords="offset points",
                     ha='center', va='bottom')
        
    # Plot the bar chart for profits of Firm 1 and Firm 2
    bars2 = ax2.bar(['Firm 1', 'Firm 2'],  [pi1, pi2], color=['blue', 'green'])
    ax2.set_ylabel('Profit')
    ax2.set_title('Profits of Firm 1 and Firm 2')
    # Set y-axis lower limit to one higher than the maximum of q1 and q2
    ax2.set_ylim([0, max(pi1, pi2) + 1])

    for bar in bars2:
        height = bar.get_height()
        ax2.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3),  # 3 points vertical offset
                     textcoords="offset points",
                     ha='center', va='bottom')
    
    # Calculate the bottom positions for stacking q1 with s
    bottom_positions = [0, s]

# Plot q1 stacked with s in two different colors
    ax1.bar('Firm 1', s, bottom=bottom_positions[0], color='black', label='s')
    ax1.bar('Firm 1', q1, bottom=bottom_positions[1], color='blue', label='q1')
    ax1.legend(loc='upper left')

    # Add text label showing total height at the top of the bar
    total_height = q1 + s
    ax1.annotate(f'{total_height:.2f}', xy=('Firm 1', total_height),
                 xytext=(0, 3),  # 3 points vertical offset
                 textcoords="offset points",
                 ha='center', va='bottom')
    
    plt.show()
    print(f"Unit Price: {price}")

# Create interactive sliders for changing initial values of a, c1, c2, and s
interact(plot_quantity_bar_chart2, initial_a=widgets.FloatSlider(value=14, min=0, max=20, step=1), 
         initial_c1=widgets.FloatSlider(value=5, min=0, max=10, step=1),
         initial_c2=widgets.FloatSlider(value=4, min=0, max=10, step=1),
         initial_s=widgets.FloatSlider(value=4, min=0, max=20, step=1))


interactive(children=(FloatSlider(value=14.0, description='initial_a', max=20.0, step=1.0), FloatSlider(value=…

<function __main__.plot_quantity_bar_chart2(initial_a, initial_c1, initial_c2, initial_s)>

# Conclusion

The analysis of the original Stackelberg game shows that the first-moving firm has an advantage and will produce more and have more profits if the firms have equal marginal costs. However, the marginal costs of the firm seems to have a larger effect on profits than the being the first mover. If Firm 2 has a cost of 1 less than that of Firm 1 then Firm 2's profit will either the very close to Firm 1's or larger.

The extension of the model looks at how the economy is affected by Firm 1 having a pre-made stock of the good. The analysis finds that if the stock is smaller than what Firm 1 would have produced the it only affects the profit of Firm 1 positively. This is because the economy is similar to the one before but Firm 1 saves the cost of producing the stock and thus enjoys higher profits. However, if Firm 1's stock is larger than what it otherwise would have produced, then it increases it's own profits but hurts Firm 2 who will produce less and get a lower price. If the stock is large enough, Firm 2 will produce nothing and have no profits.

This analysis shows that it could actually be advantageous for a company to have a larger stock in order to scare off competitors. 